This notebook uses deep learning (CNN, GRU, LSTM) to predict user ratings of movies based on user reviews. 

In [1]:
import pandas as pd
import numpy as np
import json
from time import time
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import text_to_word_sequence

Using TensorFlow backend.
/home/flip/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/flip/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/flip/.conda/envs/py36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/flip/.conda/envs/py36/lib/python3.6

Data pre-processing.

In [2]:
def get_list_of_dicts(fname): 
    return [json.loads(i) for i in open(fname, "rt")]

raw_data = get_list_of_dicts("Amazon_Instant_Video_5.json")
data = pd.DataFrame(raw_data).loc[:, ["reviewerID", "reviewText", "asin", "overall"]]

data = data[:10000] # only 10 000 user reviews

data.head()

,reviewerID,reviewText,asin,overall
0,A11N155CW1UV02,I had big expectations because I love English ...,B000H00VBQ,2.0
1,A3BC8O2KCL29V2,I highly recommend this series. It is a must f...,B000H00VBQ,5.0
2,A60D5HQFOTSOM,This one is a real snoozer. Don't believe anyt...,B000H00VBQ,1.0
3,A1RJPIGRSNX4PW,Mysteries are interesting. The tension betwee...,B000H00VBQ,4.0
4,A16XRPF40679KG,"This show always is excellent, as far as briti...",B000H00VBQ,5.0


In [ ]:
def add_user_reviews(x):
    ur = user_reviews.loc[x["reviewerID"]].drop(x["asin"]).values.tolist()
    mr = movie_reviews.loc[x["asin"]].drop(x["reviewerID"]).values.tolist()
    x["userReviews"] = " ".join(list(map(lambda x: x[0], ur)))
    x["movieReviews"] = " ".join(list(map(lambda x: x[0], mr)))
    return x

user_item_review = data.drop("reviewText", axis=1)
user_reviews = pd.pivot_table(data, index=["reviewerID", "asin"], aggfunc=lambda x: x).drop("overall", axis=1)  
movie_reviews = pd.pivot_table(data, index=["asin", "reviewerID"], aggfunc=lambda x: x).drop("overall", axis=1)

df = user_item_review.apply(add_user_reviews, axis=1)
df.head()

In [4]:
# Train-test split 
test_size = 0.005

# get test_size percentage of users
unique_users = df.loc[:, "reviewerID"].unique()
users_size = len(unique_users)
test_idx = np.random.choice(users_size, size=int(users_size * test_size), replace=False)

# get test users
test_users = unique_users[test_idx]

# everyone else is a training user
train_users = np.delete(unique_users, test_idx)

test = df[df["reviewerID"].isin(test_users)]
train = df[df["reviewerID"].isin(train_users)]

unique_test_movies = test["asin"].unique()

# drop the movies that also appear in our test set. In order to be
# a true train/test split, we are forced to discard some data entirely
train = train.where(np.logical_not(train["asin"].isin(unique_test_movies))).dropna()

train.head()

,reviewerID,asin,overall,userReviews,movieReviews
0,A11N155CW1UV02,B000H00VBQ,2.0,,"This show always is excellent, as far as briti..."
1,A3BC8O2KCL29V2,B000H00VBQ,5.0,,I had big expectations because I love English ...
2,A60D5HQFOTSOM,B000H00VBQ,1.0,I watched this a couple of weeks ago. There ar...,I had big expectations because I love English ...
3,A1RJPIGRSNX4PW,B000H00VBQ,4.0,,I had big expectations because I love English ...
4,A16XRPF40679KG,B000H00VBQ,5.0,As many people said this show kept getting bet...,I had big expectations because I love English ...


Embed the reviews into GloVe word2vect model. 

The pre-trained GloVe model is downloadable at
https://nlp.stanford.edu/projects/glove/

In [5]:
import os.path
# functions to embed user reviews into the GloVe word2vect model
def init_embeddings_map(fname):
    with open(os.path.join("glove.6B", fname), encoding="utf8") as glove:
        return {l[0]: np.asarray(l[1:], dtype="float32") for l in
                [line.split() for line in glove]}

def get_embed_func(i_len, u_len, pad_value, embedding_map):
    def embed(row):
        sentence = row["userReviews"].split()[:u_len]
        reviews = list(map(lambda word: embedding_map.get(word)
            if word in embedding_map else pad_value, sentence))
        row["userReviews"] = reviews +[pad_value] * (u_len - len(reviews))
        sentence = row["movieReviews"].split()[:i_len]
        reviews = list(map(lambda word: embedding_map.get(word) if word in embedding_map else pad_value, sentence))
        row["movieReviews"] = reviews +[pad_value] * (i_len - len(reviews))
        return row
    return embed

print("Before Embedding")
emb_size = 50 #or 100, 200, 300
embedding_map = init_embeddings_map("glove.6b." + str(emb_size) + "d.10000.txt")
print("After Embedding")

print("Before Apply")
user_sizes = df.loc[:, "userReviews"].apply(lambda x: x.split()).apply(len)
item_sizes = df.loc[:, "movieReviews"].apply(lambda x: x.split()).apply(len)
print("After Apply")

u_ptile = 40
i_ptile = 15
u_len = int(np.percentile(user_sizes, u_ptile))
i_len = int(np.percentile(item_sizes, i_ptile))

print("Before Embedding Function")
embedding_fn = get_embed_func(i_len, u_len, np.array([0.0] * emb_size), embedding_map)
print("After Embedding Function")

print("Before Embedding Train")
train_embedded = train.apply(embedding_fn, axis=1)
test_embedded = test.apply(embedding_fn, axis=1)
print("After Embedding Train")

print(u_len, i_len) # size of input in deep neural networks, useful to set parameters
train_embedded.head()

Before Embedding
After Embedding
Before Apply
After Apply
Before Embedding Function
After Embedding Function
Before Embedding Train
After Embedding Train
47 670


,reviewerID,asin,overall,userReviews,movieReviews
0,A11N155CW1UV02,B000H00VBQ,2.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,A3BC8O2KCL29V2,B000H00VBQ,5.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,A60D5HQFOTSOM,B000H00VBQ,1.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,A1RJPIGRSNX4PW,B000H00VBQ,4.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,A16XRPF40679KG,B000H00VBQ,5.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


Deep learning models

In [6]:
import tensorflow as tf
from keras.models import Model
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import Conv1D, GRU, LSTM, MaxPooling1D, Flatten
from keras.layers import Input, Dense, Dropout
from keras.layers.merge import Add, Dot, Concatenate

In [7]:
def cnn_tower(max_len, embedding_size, hidden_size, filters=4, kernel_size=10):
        input_layer = Input(shape=(max_len, embedding_size))
        tower = Conv1D(filters=filters, kernel_size=kernel_size, activation="tanh")(input_layer)
        tower = MaxPooling1D()(tower)
        tower = Conv1D(filters=filters, kernel_size=kernel_size, activation="tanh")(tower)
        tower = MaxPooling1D()(tower)
        tower = Flatten()(tower)
        tower = Dense(hidden_size, activation="relu")(tower)
        tower = Dropout(0.4)(tower)
        return input_layer, tower
    
def CNN_model(embedding_size, hidden_size, u_len, i_len):
    inputU, towerU = cnn_tower(u_len, embedding_size, hidden_size)
    inputM, towerM = cnn_tower(i_len, embedding_size, hidden_size)
    joined = Concatenate()([towerU, towerM])
    outNeuron = Dense(1)(joined)
    dotproduct = Dot(axes=1)([towerU, towerM])
    output_layer = Add()([outNeuron, dotproduct])
        
    model = Model(inputs=[inputU, inputM], outputs=[output_layer])
    return model

hidden_size = 64

model_cnn = CNN_model(emb_size, hidden_size, u_len, i_len)
model_cnn.compile(optimizer='Adam', loss='mse')
model_cnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 47, 50)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 670, 50)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 38, 4)        2004        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 661, 4)       2004        input_2[0][0]                    
__________________________________________________________________________________________________
max_poolin

In [8]:
batch_size = 32
epochs = 20

user_reviews = np.array(list(train_embedded.loc[:, "userReviews"]))
movie_reviews = np.array(list(train_embedded.loc[:, "movieReviews"]))

train_inputs = [user_reviews, movie_reviews]
train_outputs = train_embedded.loc[:, "overall"]

tensorboard = TensorBoard(log_dir="cnn_log")
earlystop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint('cnn_weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', save_best_only=True)
train_history = model_cnn.fit(train_inputs, train_outputs, callbacks=[tensorboard, earlystop, checkpoint], 
                              validation_split=0.05, batch_size=batch_size, epochs=epochs)

model_cnn.save("cnn.h5")

Train on 4106 samples, validate on 217 samples
Epoch 1/20
4106/4106 [==============================] - 8s 2ms/step - loss: 2.3231 - val_loss: 0.9956
Epoch 2/20
4106/4106 [==============================] - 8s 2ms/step - loss: 1.4956 - val_loss: 0.6674
Epoch 3/20
4106/4106 [==============================] - 12s 3ms/step - loss: 1.4027 - val_loss: 0.7987
Epoch 4/20
4106/4106 [==============================] - 15s 4ms/step - loss: 1.3458 - val_loss: 0.7344
Epoch 5/20
4106/4106 [==============================] - 9s 2ms/step - loss: 1.2139 - val_loss: 0.7520


In [9]:
def gru_tower(max_len, embedding_size, hidden_size, rnn_hidden_size, filters=2, kernel_size=8):
        input_layer = Input(shape=(max_len, embedding_size))
        tower = GRU(rnn_hidden_size, activation="tanh")(input_layer)
        tower = Dense(hidden_size, activation="relu")(tower)
        tower = Dropout(0.4)(tower)
        return input_layer, tower
    
def GRU_model(embedding_size, hidden_size, rnn_hidden_size, u_len, i_len):
    inputU, towerU = gru_tower(u_len, embedding_size, hidden_size, rnn_hidden_size)
    inputM, towerM = gru_tower(i_len, embedding_size, hidden_size, rnn_hidden_size)
    joined = Concatenate()([towerU, towerM])
    outNeuron = Dense(1)(joined)
    dotproduct = Dot(axes=1)([towerU, towerM])
    output_layer = Add()([outNeuron, dotproduct])
        
    model = Model(inputs=[inputU, inputM], outputs=[output_layer])
    return model

hidden_size = 64
rnn_hidden_size = 64

model_gru = GRU_model(emb_size, hidden_size, rnn_hidden_size, u_len, i_len)
model_gru.compile(optimizer='Adam', loss='mse')
model_gru.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 47, 50)       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 670, 50)      0                                            
__________________________________________________________________________________________________
gru_1 (GRU)                     (None, 64)           22080       input_3[0][0]                    
__________________________________________________________________________________________________
gru_2 (GRU)                     (None, 64)           22080       input_4[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

In [10]:
batch_size = 32
epochs = 20

tensorboard = TensorBoard(log_dir="gru_log")
earlystop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint('gru_weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', save_best_only=True)
train_history = model_gru.fit(train_inputs, train_outputs, callbacks=[tensorboard, earlystop, checkpoint], 
                              validation_split=0.05, batch_size=batch_size, epochs=epochs)

model_gru.save("gru.h5")

Train on 4106 samples, validate on 217 samples
Epoch 1/20
4106/4106 [==============================] - 51s 12ms/step - loss: 3.8338 - val_loss: 0.6550
Epoch 2/20
4106/4106 [==============================] - 50s 12ms/step - loss: 1.4193 - val_loss: 0.5735
Epoch 3/20


KeyboardInterrupt: 

In [ ]:
# LSTM model
def lstm_tower(max_len, embedding_size, hidden_size, rnn_hidden_size, filters=2, kernel_size=8):
        input_layer = Input(shape=(max_len, embedding_size))
        tower = LSTM(rnn_hidden_size, activation="tanh")(input_layer)
        tower = Dense(hidden_size, activation="relu")(tower)
        tower = Dropout(0.4)(tower)
        return input_layer, tower
    
def LSTM_model(embedding_size, hidden_size, rnn_hidden_size, u_len, i_len):
    inputU, towerU = lstm_tower(u_len, embedding_size, hidden_size, rnn_hidden_size)
    inputM, towerM = lstm_tower(i_len, embedding_size, hidden_size, rnn_hidden_size)
    joined = Concatenate()([towerU, towerM])
    outNeuron = Dense(1)(joined)
    dotproduct = Dot(axes=1)([towerU, towerM])
    output_layer = Add()([outNeuron, dotproduct])
        
    model = Model(inputs=[inputU, inputM], outputs=[output_layer])
    return model


hidden_size = 64
rnn_hidden_size = 64

model_lstm = LSTM_model(emb_size, hidden_size, rnn_hidden_size, u_len, i_len)
model_lstm.compile(optimizer='Adam', loss='mse')
model_lstm.summary()

In [ ]:
batch_size = 32
epochs = 30

tensorboard = TensorBoard(log_dir="lstm_log")
earlystop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint('lstm_weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', save_best_only=True)
train_history = model_lstm.fit(train_inputs, train_outputs, callbacks=[tensorboard, earlystop, checkpoint], 
                              validation_split=0.05, batch_size=batch_size, epochs=epochs)

model_lstm.save("lstm.h5")

Prediction and errors

In [ ]:
user_reviews = np.array(list(test_embedded.loc[:, "userReviews"]))
movie_reviews = np.array(list(test_embedded.loc[:, "movieReviews"]))
test_inputs = [user_reviews, movie_reviews]

true_rating = np.array(list(test_embedded.loc[:, "overall"])).reshape((-1, 1))

predictions_cnn = model_cnn.predict(test_inputs)
predictions_gru = model_gru.predict(test_inputs)
predictions_lstm = model_lstm.predict(test_inputs)

error_cnn = np.square(predictions_cnn - true_rating)
print("Test MSE for CNN model :", np.average(error_cnn))

error_gru = np.square(predictions_gru - true_rating)
print("Test MSE for GRU model :", np.average(error_gru))

error_lstm = np.square(predictions_lstm - true_rating)
print("Test MSE for LSTM model :", np.average(error_lstm))

The data is very sparse and we need to run more epochs, and also a larger dataset. So the resulting MSE's are not so satisfying. However, we can still compare them and draw some early conclusions.

1. RNN works better than CNN. A possible reason might be that reviews are sequential data.
2. LSTM works better than GRU. More epochs will lead to better performance.